# Importing Libraries

In [1]:
import sys
import pandas as pd
import numpy as np
import math
import plotly.graph_objects as go
from pmdarima import auto_arima
from sklearn.model_selection import train_test_split
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Ignoring warnings
import warnings
warnings.filterwarnings("ignore")

# To import the main.py file
sys.path.append('../')
from python_files import main

# Data Preprocessing Functions

In [2]:
def get_data():
    confirmed_global, deaths_global, recovered_global, country_cases = main.collect_data()
    recovered = recovered_global.groupby("country").sum().T
    deaths = deaths_global.groupby("country").sum().T
    confirmed = confirmed_global.groupby("country").sum().T
    deaths.index = pd.to_datetime(deaths.index, infer_datetime_format = True)
    recovered.index = pd.to_datetime(recovered.index, infer_datetime_format = True)
    confirmed.index = pd.to_datetime(confirmed.index, infer_datetime_format = True)
    return deaths, recovered, confirmed

In [3]:
def create_data_frame(dataframe,country):
    deaths, recovered, confirmed = get_data()
    if dataframe == 'deaths':
        data = pd.DataFrame(index = deaths.index, data = deaths[country].values, columns = ["Total"])

    elif dataframe == 'recovered':
        data = pd.DataFrame(index = recovered.index, data = recovered[country].values, columns = ["Total"])

    elif dataframe == 'confirmed':
        data = pd.DataFrame(index = confirmed.index, data = confirmed[country].values, columns = ["Total"])

    data = data[(data != 0).all(1)]
    
    data['Date'] = data.index
    cols = [data.columns[-1]] + [col for col in data if col != data.columns[-1]]
    data = data[cols]   

    return data

# Graphing Functions

In [4]:
def plot_forecast(data,forecast):
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=data["Date"], y=data["Total"],   
                            mode='lines',
                            name='Up till now '))
        
    fig.add_trace(go.Scatter(x=forecast.index, y=forecast.values,   
                            mode='lines',
                            name='Prediction*'))

    fig.update_layout(title={
                'text': "Forecasted results",
                'y':0.9,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
                        template = "plotly_dark",
                        xaxis_title="Date",
                        yaxis_title="Cases",
                        legend_title="Legend ",
                        font=dict(
                                family="Arial",
                                size=15,
                                color="white"
                                )
                        )
    return fig

# Functions to Train and Test the Model

In [5]:
def find_params(train_set):
    stepwise_model = auto_arima(train_set, method='nm', start_p = 0, start_q = 0,
                               max_p = 2, max_q = 2, m = 7,
                               start_P = 0, max_P=0, start_Q=1, max_Q=1, seasonal = True,
                               d = None, D = 1, n_jobs=-1, trace = True,
                               error_action = 'ignore',  
                               suppress_warnings = True, 
                               stepwise = True)
    return stepwise_model

In [6]:
def Predict(stepwise_model,train,test):
    
    stepwise_model.fit(train)
    
    pred = stepwise_model.predict(n_periods=len(test))
    
    pred = pd.DataFrame(pred,index = test.index,columns=['Prediction'])
   
    return pred

# Error Function

In [7]:
def mape(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Function for Forecasting

In [8]:
def Future(order,seasonal_order,train,test,data):
    
    model = SARIMAX(data['Total'],  
                        order = order,  
                        seasonal_order = seasonal_order) 
    result = model.fit() 
  
    forecast = result.predict(start = len(data),  
                          end = (len(data)-1) + 14).rename('Forecast') 
    
 
    error_check = result.predict(start = len(train), end = len(train) - 1 +len(test))
    error = mape(error_check,test)
    error = error
    graph = plot_forecast(data,forecast)

    return forecast,graph,error

# Calling Function

In [9]:
def arima_predict(df_name,country):
    data = create_data_frame(df_name,country)
    train = data["Total"][:len(data)*4//5]
    test = data["Total"][len(data)*4//5:]
    model = find_params(train)
    pred = Predict(model,train,test)
    mape_error = mape(test, pred["Prediction"])
    order=model.get_params()['order']
    seasonal_order=model.get_params()['seasonal_order']
    
    forecast,graph,error = Future(order, seasonal_order, train, test,data)
    return forecast,graph,(error + np.std([error, mape_error]))



# Example

In [10]:
forecast,graph,error= arima_predict("confirmed","India")

Performing stepwise search to minimize aic
Fit ARIMA(0,2,0)x(0,1,1,7) [intercept=True]; AIC=2764.882, BIC=2774.289, Time=0.345 seconds
Fit ARIMA(0,2,0)x(0,1,0,7) [intercept=True]; AIC=2784.461, BIC=2790.733, Time=0.106 seconds
Fit ARIMA(1,2,0)x(0,1,0,7) [intercept=True]; AIC=2783.123, BIC=2792.531, Time=0.175 seconds
Fit ARIMA(0,2,1)x(0,1,1,7) [intercept=True]; AIC=2763.488, BIC=2776.032, Time=0.288 seconds
Fit ARIMA(0,2,0)x(0,1,0,7) [intercept=False]; AIC=2783.183, BIC=2786.319, Time=0.068 seconds
Fit ARIMA(0,2,1)x(0,1,0,7) [intercept=True]; AIC=2778.708, BIC=2788.115, Time=0.178 seconds
Fit ARIMA(1,2,1)x(0,1,1,7) [intercept=True]; AIC=2773.038, BIC=2788.717, Time=0.225 seconds
Fit ARIMA(0,2,2)x(0,1,1,7) [intercept=True]; AIC=2755.431, BIC=2771.110, Time=0.350 seconds
Fit ARIMA(0,2,2)x(0,1,0,7) [intercept=True]; AIC=2768.797, BIC=2781.340, Time=0.144 seconds
Fit ARIMA(1,2,2)x(0,1,1,7) [intercept=True]; AIC=2759.981, BIC=2778.796, Time=0.269 seconds
Total fit time: 2.158 seconds


In [11]:
print("PREDICTIONS FOR THE NEXT 14 DAYS : \n")
for i in range(len(forecast)):
    print(forecast.index[i] , "---->", int(forecast.values[i]))
  

PREDICTIONS FOR THE NEXT 14 DAYS : 

2020-09-10 00:00:00 ----> 4560662
2020-09-11 00:00:00 ----> 4656806
2020-09-12 00:00:00 ----> 4755126
2020-09-13 00:00:00 ----> 4851708
2020-09-14 00:00:00 ----> 4939063
2020-09-15 00:00:00 ----> 5035052
2020-09-16 00:00:00 ----> 5139831
2020-09-17 00:00:00 ----> 5242983
2020-09-18 00:00:00 ----> 5347542
2020-09-19 00:00:00 ----> 5454276
2020-09-20 00:00:00 ----> 5559272
2020-09-21 00:00:00 ----> 5655041
2020-09-22 00:00:00 ----> 5759445
2020-09-23 00:00:00 ----> 5872638


In [12]:
graph.show()

In [13]:
print("ALLOW AN ERROR OF UPTO (Based on the data at hand) :",error,"%")

ALLOW AN ERROR OF UPTO (Based on the data at hand) : 4.456324819028409 %
